In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gp

from copy import copy
from unidecode import unidecode
from functools import reduce
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from bias_correction import BiasCorrection, XBiasCorrection

relevant_municipalities =  ["buenos aires bahia blanca", "buenos aires coronel pringles", "buenos aires hipolito yrigoyen", "buenos aires coronel dorrego", "buenos aires coronel suarez", "buenos aires saavedra", "santa fe san justo"]

column_names = {
    "X":"lon",
    "Y":"lat",
    "M":"member",
    "L":"lead",
    "S":"init_date",
    "Z":"height",
    "tref":"tmean",
    "prec":"rain"
}

%load_ext autoreload
%autoreload 2

### 1. Read data

**NMME**

The North [American Multi-Model Ensemble](https://www.cpc.ncep.noaa.gov/products/NMME/) consists of six Seasonal climate models whose hindcasts are available [here](http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/). We collected data from four seasonal climate models.

- NCAR/University of Miami CCSM4.0 (**CCSM4**)
- GFDL-SPEAR (**GFDL**)
- NASA Goddard Space Flight Center (GSFC) GEOS5 (**GSFC**)
- NOAA NCEP CFSv2 (**NCEP**)
- NCAR CESM - data for 2011-2015 not available; not considered in this study
- Environment Canada CanCM3 and CanCM4 - collected from Copernicus Climate Data Store; different notebook	


In [2]:
all_models = []
#for model in ["GFDL", "GFDL2", "CCSM4", "NASA", "NCEP"]:
for model in ["NCEP"]:
    path = "data/" + model + "/"
    directory = os.fsencode(path)
    prec_temp_current_model = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith("nc"):
            df = xr.open_dataset("{}{}".format(path, filename), engine="netcdf4", decode_times=False).to_dataframe().reset_index()
            ref_date = date(1960, 1, 1)
            df["S"] = df["S"].apply(lambda x: ref_date + relativedelta(months=+x))
            df.columns = [column_names[c] for c in df.columns]
            if "tmean" in df.columns:
                df = df.groupby(["lat", "lon", "init_date", "lead"]).agg({"tmean":"mean"}).reset_index()
                df["tmean"] = df["tmean"].apply(lambda x: x - 273.15) # conversion to degree celsius
            else:
                df = df.groupby(["lat", "lon", "init_date", "lead"]).agg({"rain": lambda x: np.mean(x) / 1000}).reset_index()
            
            df["init_date"] = pd.to_datetime(df["init_date"])
            df["year"] = df["init_date"].dt.year
            df["init_month"] = df["init_date"].dt.month

            df = df.loc[(df["year"] >= 1993) & (df["init_month"].between(6, 11)) & ((df["init_month"] + df["lead"]).between(6, 12.5))].reset_index(drop=True)
            prec_temp_current_model.append(df)
    
    complete_model = reduce(lambda x, y: pd.merge(x, y, on=["lat", "lon", "init_date", "lead", "year", "init_month"], how="outer"), prec_temp_current_model).assign(model=model)
    if "rain_x" in complete_model.columns:
        complete_model["rain"] = complete_model["rain_x"].fillna(complete_model["rain_y"])
        complete_model = complete_model.drop(columns={"rain_x", "rain_y"})
    if "tmean_x" in complete_model.columns:
        complete_model["tmean"] = complete_model["tmean_x"].fillna(complete_model["tmean_y"])
        complete_model = complete_model.drop(columns={"tmean_x", "tmean_y"})
    
    complete_model = gp.GeoDataFrame(complete_model, geometry=gp.points_from_xy(complete_model["lon"], complete_model["lat"]), crs="EPSG:4326") 
    all_models.append(complete_model)      
          
nmme = pd.concat(all_models, axis=0, ignore_index=False)
nmme.head()

data (16).nc


,lat,lon,init_date,lead,tmean,year,init_month,model,geometry
0,-42.0,-66.0,1993-06-01,0.5,5.864862,1993,6,NCEP,POINT (-66.00000 -42.00000)
1,-42.0,-66.0,1993-06-01,1.5,5.067041,1993,6,NCEP,POINT (-66.00000 -42.00000)
2,-42.0,-66.0,1993-06-01,2.5,6.644220,1993,6,NCEP,POINT (-66.00000 -42.00000)
3,-42.0,-66.0,1993-06-01,3.5,9.948846,1993,6,NCEP,POINT (-66.00000 -42.00000)
4,-42.0,-66.0,1993-06-01,4.5,12.998682,1993,6,NCEP,POINT (-66.00000 -42.00000)


In [22]:
nmme.loc[nmme["init_month"] == 6].head(15)

,lat,lon,init_date,lead,tmean,year,init_month,model,geometry
0,-42.0,-66.0,1993-06-01,0.5,5.864862,1993,6,NCEP,POINT (-66.00000 -42.00000)
1,-42.0,-66.0,1993-06-01,1.5,5.067041,1993,6,NCEP,POINT (-66.00000 -42.00000)
2,-42.0,-66.0,1993-06-01,2.5,6.644220,1993,6,NCEP,POINT (-66.00000 -42.00000)
3,-42.0,-66.0,1993-06-01,3.5,9.948846,1993,6,NCEP,POINT (-66.00000 -42.00000)
4,-42.0,-66.0,1993-06-01,4.5,12.998682,1993,6,NCEP,POINT (-66.00000 -42.00000)
5,-42.0,-66.0,1993-06-01,5.5,16.547327,1993,6,NCEP,POINT (-66.00000 -42.00000)
6,-42.0,-66.0,1993-06-01,6.5,19.598322,1993,6,NCEP,POINT (-66.00000 -42.00000)
27,-42.0,-66.0,1994-06-01,0.5,6.667841,1994,6,NCEP,POINT (-66.00000 -42.00000)
28,-42.0,-66.0,1994-06-01,1.5,5.984888,1994,6,NCEP,POINT (-66.00000 -42.00000)
29,-42.0,-66.0,1994-06-01,2.5,7.432153,1994,6,NCEP,POINT (-66.00000 -42.00000)


### 2. Basic Preprocessing

In [14]:
nmme.loc[nmme["model"] == "GFDL2", "model"] = "GFDL"
nmme = nmme.drop_duplicates().reset_index(drop=True)
# lead_time_1 = (2 * lead_time_0.5 + lead_time_1.5) / 2 ; lead_time_2 = (lead_time_1.5 + lead_time_2.5) / 2 ; ...
nmme["modified rain"] = nmme[["lead", "rain"]].apply(lambda x: 2*x[1] if x[0] == 0.5 else x[1], axis=1)
nmme["modified rain"] = nmme.sort_values(by=["model", "lat", "lon", "year", "init_month", "lead"])["modified rain"].rolling(2).mean()
nmme["modified tmean"] = nmme.sort_values(by=["model",  "lat", "lon", "year", "init_month", "lead"])["tmean"].rolling(2).mean()
# window size two of rolling mean means that first entry is irrelevant
nmme = nmme.loc[(nmme["lead"] != 0.5)].reset_index(drop=True)
# rename lead from X.5 to X
nmme["lead"] = nmme.loc[:, "lead"].astype(str).str[:-2].astype(int)
# adjustment of column names
nmme = nmme[["model", "year", "init_month", "lead", "lat", "lon", "modified rain", "modified tmean"]]
nmme.loc[:, "forecasted_month"] = nmme.loc[:, "init_month"] + nmme.loc[:, "lead"] - 1
nmme = nmme.rename(columns={"modified rain":"rain", "modified tmean":"tmean"})

nmme = nmme[["model", "year", "init_month", "forecasted_month", "rain", "tmean", "lat", "lon",]]
nmme = nmme.loc[nmme["forecasted_month"].between(9,11)].reset_index(drop=True)
nmme.head()

type: "['rain'] not in index"

## EXPORT

In [6]:
nmme.to_csv("data/nmme_hindcasts.csv", index=False)